In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# O X_train e y_train são tensores com valores quaisquer, apenas para exemplificação.
X_train = torch.tensor([[[[4,5,6,7], [5,6,7,8], [8,9,10,11], [4,6,7,8]]], 
                        [[[-4,5,6,-7], [5,-6,7,8], [-8,9,-10,11], [-4,-6,-7,-8]]]]).float().to(device)
X_train.div_(8)
# y_train representa as classes correspondentes
y_train = torch.tensor([0,1]).float().to(device)

In [2]:
X_train.shape

torch.Size([2, 1, 4, 4])

In [3]:
y_train.shape

torch.Size([2])

In [ ]:
import torch.nn as nn
from torch.optim import Adam

# Aqui fazemos a definição do modelo, que é uma rede neural convolucional simples.
# A saída do modelo será um valor entre 0 e 1.
def get_model():
    model = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3), # Output: (1, 2, 2)
        nn.MaxPool2d(kernel_size=2), # Output: (1, 1, 1)
        nn.ReLU(), # Ativação ReLU
        nn.Flatten(), # Achata a saída para um vetor
        nn.Linear(1, 1), # Camada linear de saída
        nn.Sigmoid() # Ativação Sigmoid para saída entre 0 e 1
    ).to(device)
    
    loss_fn = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr=0.01)
    
    return model, loss_fn, optimizer

In [8]:
model, criterion, optimizer = get_model()

In [9]:
model

Sequential(
  (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (2): ReLU()
  (3): Flatten(start_dim=1, end_dim=-1)
  (4): Linear(in_features=1, out_features=1, bias=True)
  (5): Sigmoid()
)

In [ ]:
# Criando a função de treinamento para um conjunto de dados
def train_batch(x, y, model, optimizer, loss_fn):
    model.train()
    optimizer.zero_grad()
    predictions = model(x)
    batch_loss = loss_fn(predictions.squeeze(), y.squeeze())
    batch_loss.backward()
    optimizer.step()
    return batch_loss.item()

In [11]:
from torch.utils.data import DataLoader, TensorDataset, Dataset

trn_dl = DataLoader(TensorDataset(X_train, y_train))

In [ ]:
# Fazendo o treinamento do modelo
for epoch in range(2000):
    for ix, batch in enumerate(trn_dl):
        x, y = batch
        x, y = x.to(device), y.to(device)
        loss = train_batch(x, y, model, optimizer, criterion)
        if ix % 100 == 0:
            print(f"Epoch {epoch}, Batch {ix}, Loss: {loss:.4f}")

Epoch 0, Batch 0, Loss: 0.6369
Epoch 1, Batch 0, Loss: 0.6328
Epoch 2, Batch 0, Loss: 0.6322
Epoch 3, Batch 0, Loss: 0.6322
Epoch 4, Batch 0, Loss: 0.6324
Epoch 5, Batch 0, Loss: 0.6328
Epoch 6, Batch 0, Loss: 0.6332
Epoch 7, Batch 0, Loss: 0.6337
Epoch 8, Batch 0, Loss: 0.6342
Epoch 9, Batch 0, Loss: 0.6347
Epoch 10, Batch 0, Loss: 0.6352
Epoch 11, Batch 0, Loss: 0.6357
Epoch 12, Batch 0, Loss: 0.6363
Epoch 13, Batch 0, Loss: 0.6368
Epoch 14, Batch 0, Loss: 0.6373
Epoch 15, Batch 0, Loss: 0.6379
Epoch 16, Batch 0, Loss: 0.6384
Epoch 17, Batch 0, Loss: 0.6390
Epoch 18, Batch 0, Loss: 0.6395
Epoch 19, Batch 0, Loss: 0.6400
Epoch 20, Batch 0, Loss: 0.6406
Epoch 21, Batch 0, Loss: 0.6411
Epoch 22, Batch 0, Loss: 0.6416
Epoch 23, Batch 0, Loss: 0.6421
Epoch 24, Batch 0, Loss: 0.6426
Epoch 25, Batch 0, Loss: 0.6432
Epoch 26, Batch 0, Loss: 0.6437
Epoch 27, Batch 0, Loss: 0.6442
Epoch 28, Batch 0, Loss: 0.6447
Epoch 29, Batch 0, Loss: 0.6451
Epoch 30, Batch 0, Loss: 0.6456
Epoch 31, Batch 0,

In [ ]:
model(X_train[:1])
# tensor([[0.5001]], device='cuda:0', grad_fn=<SigmoidBackward0>)
# O valor de saída está próximo de 0.5, indicando que o modelo não está muito confiante em sua previsão para a primeira amostra.
# Isso é esperado, pois o modelo foi treinado com apenas duas amostras e pode não ter aprendido padrões significativos ainda.

tensor([[0.5001]], device='cuda:0', grad_fn=<SigmoidBackward0>)